In [1]:
# check version
from pycaret.utils import version
version()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def aqi_bucket(aqi):
    if aqi <= 50: return 'Good'
    elif aqi <= 100: return 'Moderate'
    elif aqi <= 150: return 'Unhealthy for Sensitive'
    elif aqi <= 200: return 'Unhealthy'
    elif aqi <= 300: return 'Very Unhealthy'
    else: return 'Hazardous'

In [3]:
data = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/add_AQI.csv')
data['AQI_Bucket'] = data['AQI'].apply(aqi_bucket)
data = data.drop(columns=['time', 'ID', 'pm25', 'AQI', 'AQI_Class'])
data

,lat,lon,SQRT_SEA_DEM_LAT,WSPD,WDIR,TMP,TX,TN,TP,RH,PRES2M,NDVI,CO,HCHO,NO2,SO2,CLOUD,O3,AAI,AQI_Bucket
0,21.049750,105.741870,5.922647,1.136119,145.942749,20.811243,23.219995,18.939997,0.3750,74.750000,102197.203125,0.000551,0.045586,0.000060,0.000072,0.000035,0.711612,0.101653,-0.836203,Unhealthy
1,21.015250,105.800130,4.307231,1.136119,145.942749,20.811243,23.219995,18.939997,0.3750,74.750000,102197.203125,-0.003006,0.041913,0.000107,0.000086,0.000056,0.761720,0.103635,-0.718537,Unhealthy
2,21.024347,106.017288,4.988467,0.651509,145.395233,20.677492,23.029993,18.879999,0.1250,73.687500,102228.796875,0.004388,0.042329,0.000108,0.000071,0.000026,0.867689,0.102144,-0.942304,Unhealthy
3,21.023532,105.853941,4.865087,1.136119,145.942749,20.811243,23.219995,18.939997,0.3750,74.750000,102197.203125,-0.001733,0.041913,0.000097,0.000085,0.000079,0.779294,0.102875,-0.692613,Unhealthy
4,21.049750,105.741870,5.922647,2.744283,147.084442,22.176249,25.119989,20.299982,0.1875,78.012497,102025.328125,0.016035,0.042298,0.000115,0.000082,-0.000345,0.705396,0.103488,-1.119681,Unhealthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,21.024347,106.017288,4.988467,1.669253,22.916901,16.846243,18.989985,14.730005,0.0625,65.837502,102317.015625,-0.002742,0.044463,0.000133,0.000080,0.000107,1.000000,0.113309,-0.477924,Moderate
11504,20.927000,106.314000,4.784339,1.322285,353.486359,17.187494,18.839991,15.249994,0.0000,63.812500,102335.617188,0.001342,0.047781,0.000043,0.000096,0.000121,1.000000,0.113023,-0.511212,Moderate
11505,20.977400,106.430800,4.589203,0.235641,347.438232,17.732492,19.739985,15.609980,0.0000,65.050003,102115.812500,0.010294,0.047366,0.000045,0.000073,0.000172,1.000000,0.112750,-0.629895,Good
11506,21.338470,105.367300,5.228505,1.295622,97.865257,16.122246,17.824003,14.982986,0.0000,70.632500,101743.898438,-0.003163,0.049873,0.000110,0.000077,0.000105,1.000000,0.112451,-0.337096,Moderate


In [ ]:
from pycaret.classification import *
clf1 = setup(data, target = 'AQI_Bucket', silent=True, session_id=123, fold = 3)

In [5]:
loaded_bestmodel = load_model('best-model-rf')

Transformation Pipeline and Model Successfully Loaded


In [9]:
# # Lấy dữ liệu test đã được tách ra
# test_data = get_config('X_test')
# test_labels = get_config('y_test')

# # In ra dữ liệu test
# print(test_data.head())
# print(test_labels.head())


In [11]:
# Lấy dữ liệu train
train_data = get_config('X_train')
train_labels = get_config('y_train')

# Gộp đặc trưng với nhãn thực tế
train_full = train_data.copy()
train_full['Actual_Label'] = train_labels

# Lưu ra file CSV
train_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/train.csv', index=False)


In [12]:
# Lấy dữ liệu validation
val_data = get_config('X_test')  # same as test
val_labels = get_config('y_test')

# Gộp đặc trưng với nhãn thực tế
val_full = val_data.copy()
val_full['Actual_Label'] = val_labels

# Dự đoán trên dữ liệu validation
val_preds = predict_model(loaded_bestmodel, data=val_data)

# Thêm nhãn dự đoán và xác suất
val_full['Predicted_Label'] = val_preds['Label']
val_full['Score'] = val_preds['Score']

# Lưu ra file CSV
val_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/validation.csv', index=False)


[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


In [ ]:
# 1. Lấy dữ liệu test
test_data = get_config('X_test')
test_labels = get_config('y_test')

# 2. Gộp đặc trưng với nhãn thực tế
test_full = test_data.copy()
test_full['Actual_Label'] = test_labels

# 3. Dự đoán trên dữ liệu test
predictions = predict_model(loaded_bestmodel, data=test_data)

# 4. Thêm nhãn dự đoán và xác suất dự đoán vào test_full
test_full['Predicted_Label'] = predictions['Label']
test_full['Score'] = predictions['Score']

# 5. Lưu ra file CSV
test_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/test.csv', index=False)

# Kiểm tra nhanh
print(test_full.head())


[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
             lat         lon  SQRT_SEA_DEM_LAT      WSPD        WDIR  \
748    21.163853  105.923561          6.000813  4.620680   30.317810   
6542   21.187340  106.074295          5.014936  4.157365  150.475220   
10561  21.187340  106.074295          5.014936  2.569136  137.445023   
4616   21.1277

In [7]:
pred_holdouts = predict_model(loaded_bestmodel)
pred_holdouts.head()

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.6991,0,0.4820,0.6896,0.6890,0.5395,0.5430


,lat,lon,SQRT_SEA_DEM_LAT,WSPD,WDIR,TMP,TX,TN,TP,RH,...,NDVI,CO,HCHO,NO2,SO2,CLOUD,O3,AAI,AQI_Bucket,Label
0,21.163853,105.923561,6.000813,4.620680,30.317810,15.333742,17.709986,13.489985,0.000000,49.099998,...,0.045161,0.080435,0.000317,0.000056,-0.000389,0.187030,0.112409,0.118618,Moderate,Unhealthy for Sensitive
1,21.187340,106.074295,5.014936,4.157365,150.475220,21.374998,27.179987,16.540003,0.000000,66.800003,...,0.033848,0.058313,0.000072,0.000092,0.000007,0.738616,0.104847,0.100007,Unhealthy,Unhealthy
2,21.187340,106.074295,5.014936,2.569136,137.445023,28.562510,33.450005,24.950006,0.000000,72.162498,...,0.230139,0.036548,0.000229,0.000056,0.000015,0.448591,0.123157,-0.737085,Moderate,Moderate
3,21.127739,105.889542,5.476497,4.775660,356.188110,22.019312,22.851994,21.311989,43.293751,91.831253,...,0.005140,0.046584,0.000004,0.000038,-0.000340,1.000000,0.113254,-0.286214,Good,Good
4,21.037727,106.146805,5.693217,1.361999,66.469971,22.162504,24.350000,20.040003,0.625000,83.350006,...,0.017799,0.067815,0.000200,0.000064,-0.000044,0.374236,0.121116,-1.238526,Moderate,Moderate


In [14]:
#Test fold 0:
# 1. Lấy dữ liệu test
# test_data = get_config('X_test')
# test_labels = get_config('y_test')

# 2. Gộp đặc trưng với nhãn thực tế
testgoc = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_0/test.csv')

test_data = testgoc.drop(columns=['AQI_Bucket'])
test_full = test_data.copy()
test_full['Actual_Label'] = testgoc['AQI_Bucket']

# 3. Dự đoán trên dữ liệu test
predictions = predict_model(loaded_bestmodel, data=test_data)

# 4. Thêm nhãn dự đoán và xác suất dự đoán vào test_full
test_full['Predicted_Label'] = predictions['Label']
test_full['Score'] = predictions['Score']

# 5. Lưu ra file CSV
test_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_0/infer.csv', index=False)

# Kiểm tra nhanh
print(test_full.head())


[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
         lat         lon  SQRT_SEA_DEM_LAT      WSPD        WDIR        TMP  \
0  21.200344  105.948544          6.696101  2.569136  137.445023  28.562510   
1  21.123247  106.288329          5.527568  2.631024  143.305298  28.243757   
2  21.187340  106.074296          5.014936  2.569136  137.445023 

In [17]:
# 1. Lấy dữ liệu test
testgoc = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_0/test.csv')

test_data = testgoc.drop(columns=['AQI_Bucket'])
test_full = test_data.copy()
test_full['Actual_Label'] = testgoc['AQI_Bucket']

# 2. Dự đoán trên dữ liệu test
predictions = predict_model(loaded_bestmodel, data=test_data)

# 3. Thêm nhãn dự đoán và xác suất dự đoán vào test_full
test_full['Predicted_Label'] = predictions['Label']
test_full['Score'] = predictions['Score']

# 4. Tính toán các chỉ số đánh giá
from sklearn.metrics import classification_report, accuracy_score

# Chuyển đổi nhãn thực tế và nhãn dự đoán thành danh sách
actual_labels = test_full['Actual_Label']
predicted_labels = test_full['Predicted_Label']

# In báo cáo phân loại
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels))

# Tính và in độ chính xác
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# 5. Lưu ra file CSV
test_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_0/infer.csv', index=False)

# Kiểm tra nhanh
print(test_full.head())

# # Dự đoán trên dữ liệu holdout
# pred_holdouts = predict_model(loaded_bestmodel)
# print(pred_holdouts.head())

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
Classification Report:
                         precision    recall  f1-score   support

                   Good       0.92      0.88      0.90       120
              Hazardous       0.00      0.00      0.00         1
               Moderate       0.80      0.93      0.86       117
              Unhe

In [18]:
# 1. Lấy dữ liệu test
testgoc = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_1/test.csv')

test_data = testgoc.drop(columns=['AQI_Bucket'])
test_full = test_data.copy()
test_full['Actual_Label'] = testgoc['AQI_Bucket']

# 2. Dự đoán trên dữ liệu test
predictions = predict_model(loaded_bestmodel, data=test_data)

# 3. Thêm nhãn dự đoán và xác suất dự đoán vào test_full
test_full['Predicted_Label'] = predictions['Label']
test_full['Score'] = predictions['Score']

# 4. Tính toán các chỉ số đánh giá
from sklearn.metrics import classification_report, accuracy_score

# Chuyển đổi nhãn thực tế và nhãn dự đoán thành danh sách
actual_labels = test_full['Actual_Label']
predicted_labels = test_full['Predicted_Label']

# In báo cáo phân loại
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels))

# Tính và in độ chính xác
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# 5. Lưu ra file CSV
test_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_1/infer.csv', index=False)

# Kiểm tra nhanh
print(test_full.head())

# # Dự đoán trên dữ liệu holdout
# pred_holdouts = predict_model(loaded_bestmodel)
# print(pred_holdouts.head())

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
Classification Report:
                         precision    recall  f1-score   support

                   Good       0.86      0.86      0.86        56
               Moderate       0.86      0.91      0.88       146
              Unhealthy       0.93      0.89      0.91        64
Unhealthy for Sens

In [19]:
# 1. Lấy dữ liệu test
testgoc = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_2/test.csv')

test_data = testgoc.drop(columns=['AQI_Bucket'])
test_full = test_data.copy()
test_full['Actual_Label'] = testgoc['AQI_Bucket']

# 2. Dự đoán trên dữ liệu test
predictions = predict_model(loaded_bestmodel, data=test_data)

# 3. Thêm nhãn dự đoán và xác suất dự đoán vào test_full
test_full['Predicted_Label'] = predictions['Label']
test_full['Score'] = predictions['Score']

# 4. Tính toán các chỉ số đánh giá
from sklearn.metrics import classification_report, accuracy_score

# Chuyển đổi nhãn thực tế và nhãn dự đoán thành danh sách
actual_labels = test_full['Actual_Label']
predicted_labels = test_full['Predicted_Label']

# In báo cáo phân loại
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels))

# Tính và in độ chính xác
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# 5. Lưu ra file CSV
test_full.to_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/pycaret/fold_2/infer.csv', index=False)

# Kiểm tra nhanh
print(test_full.head())

# # Dự đoán trên dữ liệu holdout
# pred_holdouts = predict_model(loaded_bestmodel)
# print(pred_holdouts.head())

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
Classification Report:
                         precision    recall  f1-score   support

                   Good       0.82      0.70      0.76        20
               Moderate       0.86      0.83      0.84       117
              Unhealthy       0.81      0.95      0.88       105
Unhealthy for Sens